# Goal of the project

The goal of this project is to control a 2D quadrotor to get it to perform acrobatic moves. The controllers will be designed using the iterative LQR (iLQR) algorithm.

## 2D quadrotor

The quadrotor is depicted in the following figure <img src='quadrotor.png' width="300">
The quadrotor model is written as
$$\begin{align} 
\dot{x} &= v_x\\
m \dot{v}_x &= - (u_1 + u_2) \sin \theta \\ 
\dot{y} &= v_y\\
m \dot{v}_y &= (u_1 + u_2) \cos \theta  - m g\\
\dot{\theta} &= \omega\\
I \dot{\omega} &= r (u_1 - u_2) \end{align}$$
where $x$ is the horizontal and $y$ the vertical positions of the quadrotor and $\theta$ is its orientation with respect to the horizontal plane. $v_x$ and $v_y$ are the linear velocities and $\omega$ is the angular velocity of the robot. $u_1$ and $u_2$ are the forces produced by the rotors (our control inputs). $m$ is the quadrotor mass, $I$ its moment of inertia (a scalar), $r$ is the distance from the center of the robot frame to the propellers and $g$ is the gravity constant. To denote the entire state, we will write $z = [x, v_x, y, v_y, \theta, \omega]^T$ - we will also write $u = [u_1, u_2]^T$.

The module ```quadrotor.py``` defines the class ```Quadrotor``` that provides all the useful information about the robot and methods to simulate and animate it as shown below.

Please write the answer to all questions that are not asking for code in the report (as a single pdf file). Please submit your code as runnable Jupyter Notebooks (one for each part) or as runnable python scripts.


## Part 1 - Setting up
1. Discretize the system dynamics using the Euler method seen in class - write the time discretization step as $\Delta t$ (use symbols not numbers for the mass, etc)
2. Assume that robot starts at an arbitrary position $x(0) = x_0$, $y(0) = y_0$ and $\theta(0) = 0$ with 0 velocities. Compute $u_1^*$ and $u_2^*$ such that the robot stays at this position forever after (you may test your answer using the simulation below).
3. Analyzing the system dynamics, is it possible to move in the x direction while keeping $\theta = 0$? Explain why.
4. Analyzing the system dynamics, is it possible to have the system at rest with $\theta = \frac{\pi}{2}$ (i.e. have the quadrotor in a vertical position)? Explain why.


## Part 2 - LQR to stay in place
Now that we have $u^*$ capable of keeping the robot at rest, we can design a simple controller that ensures that the robot stays in place even when pushed around by random disturbances (e.g. due to the wind). Our task here will be to design a LQR controller that keeps the robot at a predefined position. Since the dynamics is not linear, we need to compute a linear approximation of it.
1. Linearize the dynamics at an arbitrary operating point $\bar{z}$, $\bar{u}$ and write the linearized system dynamics using the variables $\hat{z}_n = z_n - \bar{z}$ and $\hat{u}_n = u_n - \bar{u}$.
2. Write a function ```get_linearization(robot, z, u)``` that returns the matrices A and B given a robot (object of type ```Quadrotor```), a state z and a control u.
3. Using the linearized dynamics, we can design an infinite horizon LQR controller of the form, $\hat{u} = K \hat{z}$ to stabilize the resting point. What will be the actual controller in the original coordinates $u$ as a function of $z$?
4. Design such an infinite-horizon LQR controller to stabilize the origin $z=0$ and test it using the simulator below (Explain your intended design in the report, including the cost function and found control law). In particular, verify that it can handle perturbations by calling the ```simulate``` function with ```disturbance = True``` (when setting disturbance to True, the simulator will generate a random perturbation every 1 second). Simulate your controller for 10 seconds, plot the state evolution and show the animation (show the plots in your report).


## Part 3 - following a trajectory using linearized dynamics
Now we want to follow a given trajectory leveraging a linearized version of the dynamics to design LQ controllers.
1. Assume that we want to follow a circle of radius 1 centered at (0,0) while keeping an orientation $\theta=0$, how does the linearization of the dynamics change along the desired trajectory? Why?
2. Design a tracking controller (using an LQ design) to follow this desired trajectory (Explain your design in the report). 
3. Test the tracking controller with the simulation (with and without the perturbations) and verify that you can indeed track the (x,y) trajectory very well. Are you able to also track $\theta$? (Explain) Analyze your results (including plots of the states, controls, etc). What benefits and issues do you see with this approach?


## Part 4 - iterative LQR
Now we would like to do more complicated motions with the robot, like a flip. In this case, we do not have a prescribed trajectory but we would like to compute a locally optimal trajectory together with the controller. We will design an iterative LQR algorithm to solve this problem.
1. For the first task, we want the robot to reach a vertical orientation $\theta = \frac{\pi}{2}$ at the location $x=3$ and $y=3$ at time $t=5$ starting from $z_0=0$. During the rest of the motion, the robot should try and stay close to the origin. It should also try to keep its control $u$ close to the control needed to keep the robot at rest. We want to make sure the robot reaches the origin $z=0$ at the end of the movement. Find a time-varying cost function that promotes such a behavior (use only quadratic/linear terms for simplicity). Assume $T=10$ seconds.
2. Write a function ```compute_cost(z,u, horizon_length)``` that returns the cost of a trajectory z with control trajectory u.
3. Compute the quadratic approximation of your cost function along a trajectory $z_n$ with control trajectory $u_n$ (this is not just your cost function!)
4. Write a function ```get_quadratic_approximation_cost(z, u, horizon_length)``` that returns the quadratic approximation (Hessian matrices and Jacobians) of the cost function when approximated along the trajectory z with control trajectory u.
5. Write the iLQR algorithm that solves the problem using the functions written above. At each iteration, for the line search, start with $\alpha = 1.$ and decrease it by half when the cost does not improve (you can stop when $\alpha < 0.01$).
6. Test the algorithm using as initial guess $u$ such that the robot is at rest (cf. Part1.2). Analyze your results (probably you will need to "tune" your cost function), plot the initial and final state and control trajectories, show the animation. (use the simulation without perturbations for simplicity). What benefits and issues do you see with this approach?
7. Use iLQR (and a new cost function) to get the quadrotor to do a full flip, trying to reach the upside-down state $x=1.3$, $y=3$ and $\theta = \pi$ at $t=5$ and upright state $x=3$, $y=3$ and $\theta = \pi$ at $t=10$. Analyze your results. What benefits and issues do you see with this approach? Could you run the resulting controller on a real robot?

In [1]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['animation.ffmpeg_path'] = r"C:\Users\tianshu\Anaconda3\envs\RLproject\Library\bin\ffmpeg.exe"

import quadrotor

In [2]:
# we create a robot
robot = quadrotor.Quadrotor()

# we can get its mass, half length (r), gravity constant
print(f'm is {robot.mass}')
print(f'r is {robot.length}')
print(f'I is {robot.inertia}')
print(f'g is {robot.g}')

# we can also get the integration step used in the simulation
print(f'dt is {robot.dt}')

# we can get the size of its state and control vector
print(f'number of states {robot.ns} and number of controls {robot.nu}')
print('the states are indexed as follows: x, vx, y, vy, theta, omega')

m is 0.5
r is 0.15
I is 0.1
g is 9.81
dt is 0.01
number of states 6 and number of controls 2
the states are indexed as follows: x, vx, y, vy, theta, omega


In [3]:
# we can simulate the robot but we need to provide a controller of the following form
def dummy_controller(state, i):
    """
        the prototype of a controller is as follows
        state is a column vector containing the state of the robot
        i is the index corresponding to the time step in the horizon (useful to index gains K for e.g.)
        
        this controller needs to return an array of size (2,)
    """
    # here we do nothing and just return some non-zero control
    return 1. * np.ones([2,])




# we can now simulate for a given number of time steps - here we do 10 seconds
horizon_length = 1000
z0 = np.zeros([robot.ns,])
t, state, u = robot.simulate(z0, dummy_controller, horizon_length, disturbance = False)

In [4]:
# we can plot the results
plt.figure(figsize=[9,6])

plt.subplot(2,3,1)
plt.plot(t, state[0,:])
plt.legend(['X'])

plt.subplot(2,3,2)
plt.plot(t, state[2,:])
plt.legend(['Y'])

plt.subplot(2,3,3)
plt.plot(t, state[4,:])
plt.legend(["theta"])

plt.subplot(2,3,4)
plt.plot(t, state[1,:])
plt.legend(['Vx'])
plt.xlabel('Time [s]')

plt.subplot(2,3,5)
plt.plot(t, state[3,:])
plt.legend(['Vy'])
plt.xlabel('Time [s]')

plt.subplot(2,3,6)
plt.plot(t, state[5,:])
plt.legend(['omega'])
plt.xlabel('Time [s]')

# we can also plot the control
plt.figure()
plt.plot(t[:-1], u.T)
plt.legend(['u1', 'u2'])
plt.xlabel('Time [s]')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Text(0.5, 0, 'Time [s]')

In [5]:
# now we can also create an animation
robot.animate_robot(state, u)

<source type="video/mp4" src="data:video/mp4;base64,AAAAIGZ0eXBNNFYgAAACAE00ViBpc29taXNvMmF2YzEAAAAIZnJlZQAAKjdtZGF0AAACoQYF//+d
3EXpvebZSLeWLNgg2SPu73gyNjQgLSBjb3JlIDE2MSAtIEguMjY0L01QRUctNCBBVkMgY29kZWMg
LSBDb3B5bGVmdCAyMDAzLTIwMjAgLSBodHRwOi8vd3d3LnZpZGVvbGFuLm9yZy94MjY0Lmh0bWwg
LSBvcHRpb25zOiBjYWJhYz0xIHJlZj0zIGRlYmxvY2s9MTowOjAgYW5hbHlzZT0weDM6MHgxMTMg
bWU9aGV4IHN1Ym1lPTcgcHN5PTEgcHN5X3JkPTEuMDA6MC4wMCBtaXhlZF9yZWY9MSBtZV9yYW5n
ZT0xNiBjaHJvbWFfbWU9MSB0cmVsbGlzPTEgOHg4ZGN0PTEgY3FtPTAgZGVhZHpvbmU9MjEsMTEg
ZmFzdF9wc2tpcD0xIGNocm9tYV9xcF9vZmZzZXQ9LTIgdGhyZWFkcz0xMiBsb29rYWhlYWRfdGhy
ZWFkcz0yIHNsaWNlZF90aHJlYWRzPTAgbnI9MCBkZWNpbWF0ZT0xIGludGVybGFjZWQ9MCBibHVy
YXlfY29tcGF0PTAgY29uc3RyYWluZWRfaW50cmE9MCBiZnJhbWVzPTMgYl9weXJhbWlkPTIgYl9h
ZGFwdD0xIGJfYmlhcz0wIGRpcmVjdD0xIHdlaWdodGI9MSBvcGVuX2dvcD0wIHdlaWdodHA9MiBr
ZXlpbnQ9MjUwIGtleWludF9taW49MTAgc2NlbmVjdXQ9NDAgaW50cmFfcmVmcmVzaD0wIHJjX2xv
b2thaGVhZD00MCByYz1jcmYgbWJ0cmVlPTEgY3JmPTIzLjAgcWNvbXA9MC42MCBxcG1pbj0wIHFw
bWF4PTY5IHFwc3RlcD00IGlwX3JhdGlvPTEuNDAgYXE9MToxLjAwAIAAABHdZYiEABH//veIHzLL
afk613IR560urR9Q7kZxXqS9/iAAAAMAAAMAAAMACpJYniKH8XjN/AAAAwAAt4AXsAwgCqgMKj6I
dndnTzAIiR6+OnDbVkTjpPexGHLjoXY30hkXEhSE9kEKIJNbrApCI4G73ihHZi4T1K1iog5Wc0p6
pLHThCwCmnYhSqYpYbTo2n5tZjl9Sersc3rmxTCU1aG5ejWlf2ZiBPtJ6vuZEf6FCWPekrjf3iOx
g9nYeAC3//ekaJ6Uup8pEnX74AtF5miPCXmGYp1VgIlMhblKE1f5XLlxb1HUXmuG+fEgDCAJNggr
SWMoFVS8OFpaDDMvSO64pmzh6vHDoKtfmcA4i12uUsh7rwNZe/BI7I9oGTBTCygXCZI11ZLx7x+s
Vrgw6t4wizHzoUyL85pxnWhP72FWa6UKhObVpR9cC86bAlV4WjIcMipSB2FwsyOe4UYRl5OQCbO/
KC9y9LhXenP0RWuzhVf5LhSh4/pKjOAzgAvgKIAt05ER6PI+SaU7yokDOJyww/lwIPTXg+lvHHHt
pfxUetiFUyWmZqWV7Vj9z9wT/ALctZL9/HHrp2ErRcgP9OS/IT56XLfBsAlpc07Wwr00XhByc9Wu
YLaUwATT8vv7QrrzEejXrKVhtKewskbi5hmxa7pmsTSJBvSpFZz0TZh4HejY2uXzEVy/ys1ILnvQ
HFPuEeR3nyInnqy0KIuiVhnyZNFPzP+K/m1YO/ZMefz2Rv7CC47TebMb/raY/z534x6/zeuNvDif
F3v4sf7PBrPmO4x1/8e9czRIOazYItmIqjtP6pqSv28FUlng3OCJo61udemm/ifRtYvolB4BcreO
pOb/KCPQvdHIQ8gYRz150VuCTUWVzLIRiNJmk8NQRlrbpVybXsImX+0aYRc3iyN+poCEsJgILUH2
D4Xkcp5pbhRDUwAAAwAAAwAAAwAABP4rZZQCtNpZIqu0Ajp4M6ULjVCsrd+uAcMDpOSHv8PDGxhq
JuLpnP+1zeNjKnFSf7/D+MVytEJz6vGt+qAEfyTRDOPewWqV+MY/m/cAflfJcF9ztqIJKEhmMd4Y
kIhQvSPBKAIfv+DpYjljfC+ga/1XWUeg2OEL2jwJQhW0RpR1RkHYSUUD/51lGgU/qvTcEtcZFzhD
u1RN4/+ZmB4NzAOxEYZrwFL0CIUCDtfrnMQVhM94yNfUoZHgTZu0L7w7IMIIbff1x5EyKbw2ErxZ
Bwszi+0OSmWEUAX2mJIBqiCSA2jkq3v/5FbzkzQnzRoqoyO9+Wa0uwoQhhu8IpUSAfJ3G4BiLA04
C7ZANsMLaGYwAx5/lY5OPy77JxLESp52N8+wdyZ4aSoGKY8fwSlPH+XAAAG6MwmxAAj4cqGoAK8x
ryoAzLbHPTLmukOUXIW3DenJ6QoPh8mkEFEMxGLjMOWMsBQaLHk0oVRW1BvB6jQBE4TZ0nN+XYOq
l39eHPl962no9rsn0PoytBe9aqfoG0vjwaG35408wzY5ZW10VxKgk9NgoG9FllLmRk5EDi3KeVx4
D5MYRMbYKm10Agtkm9CK1rYD4K2pbrV5Y7rdCUOzdca8j7zOTaaOUQBXzdPCvL2XfAi8nY863Gen
q27ny2bvgGfw1dlWoEKJEnZmSSb1ItRr8wRD2FxNgbSfLQ3dpOzWaMKUxugYdYM2ckNX0aG5jxaz
0bHK+9t1riYxuFSlQ3JyUL2s6BAsnO+hgxguFd3WgbiMjj66Uhy9S0/9RM+u7qPHCuSWYqQPPJid
ABhQu5KGY20H9aRj53Eyq6PGkO20sQZrD3e40xV4n3fxe/47oCD75nBK21GE6SXkg041CCPZu7T7
uPA0QdDNOS3nCSSJ65dIZ82PYB1fG1eJhBIp/yxFyJLzM0AAA/7PJgAABvopQIACxzAGCzz+cXQP
LKgXYCqDtlLpwfgE15gb8X5axT/6Z8+PKWbA+FY93IrouQb2uPc8RjibydCp/LnsfzvoOF1DdYpp
+5imm0U3J5OBIfUWsU+4IogHaQmkxc1Z+wVneezjdxkF7Cv1H9G4EZ7Tz4OXHZAf2bRU7fsnPHz/
GVCjVi1IWDGHHyR4eFTVH/KStblYYD/pjxiqU7WdKkZNWp7gN8lAw5bMgN3IiVdYitY5+fCaaXMc
xaIJBCpvVf3lwntnJxZ/vE+rwBbh/PoZak9qhnrfNmU+3qVsP9A6GRn7kg9Za1HClg29CwZvVErY
gYYMk9pnyXKGvEJUtaSZI0VRWgZGaJeN+phrxrKuULmw0eH8vvvBsYQJh0mmP39rmgaR6qaLyj9M
QwS7gFQfEkjERt+nWf/kh/HpIA7vhnLVvHPmGfQ2fBoHbjKfSvJgmLAP6Mf/6R6x4o1OqbolxCTe
FrBv0VwFGXPMzTh2O+6fL6X/fpAOlq+UkzhXW87xSCv4oGVipfuv16laJECmb9nZ5g/K38F9MADQ
Wu5Fw+6QsjwcL5uKRFDEhF/XBw+3YqMnhIyv5ptQP7q9qgO0LMXLfYd0hjI/V9ecfJZTXRCyZyPv
yf/SOZGuOeS2YNObvl5p5GlaYzdlwMv0G3qYWJP88jufuKFnq62UgTtj7mvWFIGPmbAUM3CT1j2l
9ppWvQaTDp7SV7KOR23plPyzD44lHf1lI5xrezTAAATQEP1yhKBE+YdeEZX24AAAAwAASFFm+feT
+NDZy7pzRwm9Bf+lKDM28wOY16TEkxBMqeBry4swFZGRb8woGmsu8aDrG/Roy9QEObkSso+YsW1s
NjVtXmnUco3SuNw+6ThzPw230JfszwdkE1tbOH5gQSSkdykb21P0NcG0o+6LvGQG/ejGZ8gGgdO3
njQ0j7pJ+oZb0qMbeI29B6inCUlnKtpw90LfWTxBVuPp5+aiYqhiOrb/pIqy0vA3I0P+/AHY8LuH
wuM4EJ2wQb1TnH8PZ68Lj

In [6]:
# we can also simulate with perturbations
t, state, u = robot.simulate(z0, dummy_controller, horizon_length, disturbance = True)

# we can plot the results
plt.figure(figsize=[9,6])

plt.subplot(2,3,1)
plt.plot(t, state[0,:])
plt.legend(['X'])

plt.subplot(2,3,2)
plt.plot(t, state[2,:])
plt.legend(['Y'])

plt.subplot(2,3,3)
plt.plot(t, state[4,:])
plt.legend(["theta"])

plt.subplot(2,3,4)
plt.plot(t, state[1,:])
plt.legend(['Vx'])
plt.xlabel('Time [s]')

plt.subplot(2,3,5)
plt.plot(t, state[3,:])
plt.legend(['Vy'])
plt.xlabel('Time [s]')

plt.subplot(2,3,6)
plt.plot(t, state[5,:])
plt.legend(['omega'])
plt.xlabel('Time [s]')

# we can also plot the control
plt.figure()
plt.plot(t[:-1], u.T)
plt.legend(['u1', 'u2'])
plt.xlabel('Time [s]')

robot.animate_robot(state,u)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Your browser does not support the video tag.